In [ ]:
import os, arcpy
import pandas as pd
from arcgis.features import GeoAccessor, GeoSeriesAccessor

In [ ]:
arcpy.env.overwriteOutput = True

RUTA_BASES_TERRITORIOS_GDB = r"C:\docsProyectos\6.VALORMAS\2024\5.Territorios+\zInsumos\GDB_TerritoriosMas"
BD_CONSOLIDACION = "GDB_TerritoriosMas.gdb"

DB_CONSOLIDADA = os.path.join(RUTA_BASES_TERRITORIOS_GDB, BD_CONSOLIDACION)

NOMBRE_CAMPO = "area_ha_ctm12"
PROPIEDADES_GEOMETRIA = [NOMBRE_CAMPO,'AREA']
RUTA_SISTEMA_REFERENCIA = r"C:\docsProyectos\6.VALORMAS\2024\5.Territorios+\zInsumos\GDB_TerritoriosMas\\9377.prj"

<h3>Crear BD Unificación Territorios+</h3>

In [ ]:
if arcpy.Exists(DB_CONSOLIDADA):
    print(f"La BD {BD_CONSOLIDACION} existe")
    pass
else:
    arcpy.CreateFileGDB_management(out_folder_path=RUTA_BASES_TERRITORIOS_GDB, out_name=BD_CONSOLIDACION)
    print(f"Se crea la BD {BD_CONSOLIDACION}")

<h3>Migración de capa Terrenos</h3>

In [ ]:
arcpy.env.workspace = r"C:\docsProyectos\6.VALORMAS\2024\5.Territorios+\zInsumos\GDB_TerritoriosMas\modeloRelacional_034.gdb"

for capa in arcpy.ListFeatureClasses():
    if capa == 'lc_terreno':
        print(f" - La capa {capa} existe")
        capa_terreno_origen = os.path.join(arcpy.env.workspace, capa)
        capa_terreno_destino = os.path.join(RUTA_BASES_TERRITORIOS_GDB,BD_CONSOLIDACION,capa)
        arcpy.Copy_management(capa_terreno_origen, capa_terreno_destino)
        print(f"- La capa {capa} se migra para truncado")
        arcpy.TruncateTable_management(capa_terreno_destino)
        print(f" - Se trunca la capa {capa} migrada a la ruta {capa_terreno_destino}")
    else:
        pass

<h3>Unificación de Terrenos</h3>

In [ ]:
lectura_bdTerritorios = os.listdir(RUTA_BASES_TERRITORIOS_GDB)

lista_bdTerritorios = []
for bd in lectura_bdTerritorios:
    if bd != BD_CONSOLIDACION:
        ruta_bd_a_consolidar = os.path.join(RUTA_BASES_TERRITORIOS_GDB, bd)
        lista_bdTerritorios.append(ruta_bd_a_consolidar)
        print(f" - BD {bd} consolidada en lista")

for bdc in lista_bdTerritorios:
    arcpy.env.workspace = bdc
    for capa in arcpy.ListFeatureClasses():
        if capa == 'lc_terreno':
            ruta_capa_origen = os.path.join(arcpy.env.workspace,capa)
            ruta_capa_destino = os.path.join(RUTA_BASES_TERRITORIOS_GDB,BD_CONSOLIDACION,capa)
            arcpy.Append_management(inputs=ruta_capa_origen, target=ruta_capa_destino, schema_type='NO_TEST')
            print(f" - Join entre FC {ruta_capa_origen} Y FC {ruta_capa_destino}")  


<h3>Cálculo de Hectareaje</h3>

In [ ]:
arcpy.env.workspace = DB_CONSOLIDADA

for capa in arcpy.ListFeatureClasses():
    if capa == "lc_terreno":
        arcpy.AddField_management(in_table=capa, field_name=NOMBRE_CAMPO, field_type="DOUBLE")
        print(f" - Se crea campo {NOMBRE_CAMPO} en capa {capa}")
        arcpy.management.CalculateGeometryAttributes(in_features=capa, geometry_property=[PROPIEDADES_GEOMETRIA], area_unit='HECTARES', coordinate_system=RUTA_SISTEMA_REFERENCIA)
        print(f"Se calcula el hectareaje en la capa {capa}")
        for campo in arcpy.ListFields(dataset=capa):    
            print(f"Capa {capa} - Campo {campo.name}")
    else:
        pass

<h3>Espacialización DF</h3>

In [ ]:
for capa in arcpy.ListFeatureClasses():
    if capa == "lc_terreno":
        capa_terrenos = os.path.join(arcpy.env.workspace, capa)
        df_lc_terrenos = pd.DataFrame.spatial.from_featureclass(capa_terrenos)

In [ ]:
df_lc_terrenos_por_munipio = pd.DataFrame(df_lc_terrenos['local_id'])
# ? Manejo del Slicing dentro de pandas
df_lc_terrenos_por_munipio['municipio'] = df_lc_terrenos_por_munipio['local_id'].str[:5]

df_lc_terrenos_por_munipio_conteo = df_lc_terrenos_por_munipio['municipio'].value_counts()
df_lc_terrenos_por_munipio_conteo = pd.DataFrame(df_lc_terrenos_por_munipio_conteo)
df_lc_terrenos_por_munipio_conteo = df_lc_terrenos_por_munipio_conteo.reset_index()

<h3>Total de Terrenos T+</h3>

In [ ]:
"""
isin se usa para verificar si cada valor de una columna está en una lista dada.
Devuelve una Serie booleana.
Usando ~, puedes invertir la máscara para excluir los valores en la lista.
"""

valores_no_deseados = ['04410', '5', ' ', '2', '1', '3']

df_lc_terrenos_por_munipio_filtro_errores = df_lc_terrenos_por_munipio_conteo[~df_lc_terrenos_por_munipio_conteo['municipio'].isin(valores_no_deseados)]

df_lc_terrenos_por_munipio_filtro_errores

In [ ]:
df_lc_terrenos['municipio'] = df_lc_terrenos['local_id'].str[:5]

df_lc_terrenos_filtro = pd.merge(df_lc_terrenos, df_lc_terrenos_por_munipio_filtro_errores, left_on='municipio', right_on='municipio', how='inner')

<h3>Suma del Hectareaje</h3>

In [ ]:
Total = df_lc_terrenos_filtro[NOMBRE_CAMPO].sum()
print(f"La suma del Hectareaje total para T+ es de: {Total.round(2)}")